## Banning Transit is missing
[PR](https://github.com/cal-itp/data-analyses/issues/1254)

In [1]:
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from shared_utils import catalog_utils, rt_dates, rt_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS

In [2]:
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)

In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
import _operators_prep as op_prep

In [5]:
analysis_date_list = rt_dates.y2024_dates + rt_dates.y2023_dates

## 11/18: why isn't `deploy_portfolio_yaml` working

In [6]:
df = (
    op_prep.operators_schd_vp_rt()
    .sort_values(["caltrans_district", "organization_name"])
    .reset_index(drop=True)
)

In [7]:
# df

In [8]:
chapter_info = {
    "column": "caltrans_district",
    "name": "district",
    "caption_prefix": "District ",
    "caption_suffix": "",
}

In [9]:
section_info = {
    "column": "organization_name",
    "name": "organization_name",
}

In [10]:
chapter_col = chapter_info["column"]

In [11]:
chapter_col

'caltrans_district'

In [12]:
chapter_values = sorted(list(df[chapter_col].unique()))

In [14]:
import yaml
from pathlib import Path

In [15]:
PORTFOLIO_SITE_YAML = Path("../portfolio/sites/gtfs_digest.yml")

In [16]:
prefix = chapter_info["caption_prefix"]

In [17]:
section_col = section_info["column"]

In [18]:
with open(PORTFOLIO_SITE_YAML) as f:
        site_yaml_dict = yaml.load(f, yaml.Loader)

In [19]:
chapters_list = []
for district in chapter_values:
    df2 = df.loc[df.caltrans_district == district]
    chapters_list_one_district = [
    {
        **{
            "caption": f"{prefix} {district}",
            "params": {chapter_info["name"]: district},
            "section": [
                {section_info["name"]: one_section_value}
                for one_section_value in df2[section_col].unique().tolist()
            ],
        }
    }]
    chapters_list.append(chapters_list_one_district)

In [20]:
chapters_list[0]

[{'caption': 'District  01 - Eureka',
  'params': {'district': '01 - Eureka'},
  'section': [{'organization_name': 'Blue Lake Rancheria'},
   {'organization_name': 'City of Arcata'},
   {'organization_name': 'City of Eureka'},
   {'organization_name': 'Curry Public Transit'},
   {'organization_name': 'Humboldt Transit Authority'},
   {'organization_name': 'Lake Transit Authority'},
   {'organization_name': 'Mendocino Transit Authority'},
   {'organization_name': 'POINT'},
   {'organization_name': 'Redwood Coast Transit Authority'}]}]

In [21]:
parts_list = [{"chapters": chapters_list}]

In [22]:
type(parts_list)

list

In [24]:
site_yaml_dict["parts"] = parts_list

In [29]:
site_yaml_dict

{'directory': './gtfs_digest/',
 'notebook': './gtfs_digest/03_report.ipynb',
 'parts': [{'chapters': [[{'caption': 'District  01 - Eureka',
      'params': {'district': '01 - Eureka'},
      'section': [{'organization_name': 'Blue Lake Rancheria'},
       {'organization_name': 'City of Arcata'},
       {'organization_name': 'City of Eureka'},
       {'organization_name': 'Curry Public Transit'},
       {'organization_name': 'Humboldt Transit Authority'},
       {'organization_name': 'Lake Transit Authority'},
       {'organization_name': 'Mendocino Transit Authority'},
       {'organization_name': 'POINT'},
       {'organization_name': 'Redwood Coast Transit Authority'}]}],
    [{'caption': 'District  02 - Redding',
      'params': {'district': '02 - Redding'},
      'section': [{'organization_name': 'Lassen Transit Service Agency'},
       {'organization_name': 'Modoc Transportation Agency'},
       {'organization_name': 'Plumas Transit Systems'},
       {'organization_name': 'Reddin

In [25]:
output = yaml.dump(site_yaml_dict)

In [26]:
with open(PORTFOLIO_SITE_YAML, "w") as f:
        f.write(output)

## 11/18: double check the yaml generation

In [27]:
stop

NameError: name 'stop' is not defined

In [ ]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

In [ ]:
schd_vp_df = pd.read_parquet(
    schd_vp_url,
    filters=[[("sched_rt_category", "in", ["schedule_and_vp", "schedule_only"])]],
    columns=[
        "schedule_gtfs_dataset_key",
        "caltrans_district",
        "organization_name",
        "name",
        "sched_rt_category",
        "service_date",
    ],
)

In [ ]:
schd_vp_df.shape

In [ ]:
schd_vp_df.sched_rt_category.value_counts()

In [ ]:
schd_vp_df.info()

In [ ]:
schd_vp_df.loc[schd_vp_df.caltrans_district.isna()][
    ["organization_name"]
].drop_duplicates()

In [ ]:
schd_vp_df2 = (
    schd_vp_df.dropna(subset="caltrans_district")
    .sort_values(
        by=[
            "caltrans_district",
            "organization_name",
            "service_date",
        ],
        ascending=[True, True, False],
    )
    .drop_duplicates(
        subset=[
            "organization_name",
            "caltrans_district",
        ]
    )
    .reset_index(drop=True)
)

In [ ]:
schd_vp_df2.shape

In [ ]:
schd_vp_df2.sched_rt_category.value_counts()

In [ ]:
schd_vp_df2.organization_name.nunique()

In [ ]:
schd_vp_df2.service_date.value_counts()

In [ ]:
# schd_vp_df2

In [ ]:
df = (
    op_prep.operators_schd_vp_rt()
    .sort_values(["caltrans_district", "organization_name"])
    .reset_index(drop=True)
)

In [ ]:
df